In [1]:
from langchain.prompts import PromptTemplate
from langchain_core.messages.human import HumanMessage
from langchain_community.chat_models.openai import ChatOpenAI

import os
import dotenv

dotenv.load_dotenv()

from agential.cog.prompts.critic import CRITIC_INSTRUCTION_GSM8K_POT, GSM8K_FEWSHOT_EXAMPLES_POT, CRITIC_CRITIQUE_INSTRUCTION_GSM8K, GSM8K_FEWSHOT_EXAMPLES_CRITIC

llm = ChatOpenAI()

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [2]:
question = "Billy is buying some candy with $10 his father gave him. The candy costs $1.5 a pound. After buying candy, he takes half his change and spends it on gumballs, which cost $.05 each. If he bought 40 gumballs, how many pounds of candy did he buy?"

prompt = CRITIC_INSTRUCTION_GSM8K_POT.format(
    examples=GSM8K_FEWSHOT_EXAMPLES_POT,
    question=question
)

out = llm(
    [
        HumanMessage(
            content=prompt,
        )
    ]
).content

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [3]:
print(out)

total_money = 10
cost_per_pound = 1.5
cost_per_gumball = 0.05
num_gumballs = 40

money_spent_on_candy = total_money / 2
num_gumballs_bought = money_spent_on_candy / cost_per_gumball

remaining_money = total_money - money_spent_on_candy
pounds_of_candy_bought = remaining_money / cost_per_pound

answer = pounds_of_candy_bought


In [4]:
import func_timeout


def safe_execute(code_string: str, keys=None):
    def execute(x):
        try:
            exec(x)
            locals_ = locals()
            if keys is None:
                an = locals_.get('answer', None)
            else:
                an = [locals_.get(k, None) for k in keys]
            return an, "Done"
        except BaseException as e: # jump wrong case
            return None, repr(e)

    try:
        an, report = func_timeout.func_timeout(3, execute, args=(code_string,))
    except func_timeout.FunctionTimedOut:
        an = None
        report = "TimeoutError: execution timeout"

    return an, report

In [5]:
answer, status = safe_execute(out)

In [6]:
critic_prompt = CRITIC_CRITIQUE_INSTRUCTION_GSM8K.format(
    examples=GSM8K_FEWSHOT_EXAMPLES_CRITIC,
    question=question,
    answer=answer,
    status=status,
    code=out,
    critique=""
)

out_ = ""
critique = llm(
    [
        HumanMessage(
            content=critic_prompt,
        )
    ]
).content.split("Here's")[0]

In [7]:
out_ += critique
out_ += "Here's a better solution:\n```python\n"

In [8]:
print(out_)

The problem with the above code is that the number of pounds of candy bought should be a whole number, not a floating-point number. Billy cannot buy a fraction of a pound of candy. Additionally, the calculation for the number of gumballs bought is not used to determine the amount of candy bought.

Here's a better solution:
```python



In [11]:
critic_prompt_1 = CRITIC_CRITIQUE_INSTRUCTION_GSM8K.format(
    examples=GSM8K_FEWSHOT_EXAMPLES_CRITIC,
    question=question,
    answer=answer,
    status=status,
    code=out,
    critique=out_
)

critique_1 = llm(
    [
        HumanMessage(
            content=critic_prompt_1,
        )
    ]
).content

In [12]:
print(critique_1.split("```")[0])

total_money = 10
cost_per_pound = 1.5
cost_per_gumball = 0.05
num_gumballs = 40

money_spent_on_candy = total_money / 2
num_gumballs_bought = money_spent_on_candy / cost_per_gumball

remaining_money = total_money - money_spent_on_candy
pounds_of_candy_bought = remaining_money / cost_per_pound

# Since the pounds of candy bought should be a whole number, we need to round down to the nearest integer
pounds_of_candy_bought = int(pounds_of_candy_bought)

answer = pounds_of_candy_bought


In [ ]:
out_ += 